In [1]:
# Get raw data
with open('input/20.txt', 'r') as f:
    rawinput = f.read().strip()

In [2]:
import numpy as np

In [3]:
# Part 1
class RoomMap(object):
    compass = {'N': np.array([-1,0]), 
               'E': np.array([0,1]), 
               'S': np.array([1,0]), 
               'W': np.array([0,-1])}
    
    def __init__(self, instructions):
        self.instructions = instructions
        self.ptr = 0
        self.branch_pts = []
        self.layout = np.array([['.']])
        self.size = 0
        self.loc = np.array([0,0])
        self.reach = None
        
    def do_mapping(self):
        while self.ptr < len(self.instructions):
            instr = self.instructions[self.ptr]
            if instr == '(':
                self.branch_pts += [np.copy(self.loc)]
            elif instr in('|', ')'):
                self.loc = np.copy(self.branch_pts[-1])
                if instr == ')':
                    self.branch_pts = self.branch_pts[:-1]
            else:
                self.do_step(instr)
            self.ptr += 1
        
    def do_step(self, instr):
        if np.any(np.abs(self.loc + (step:=self.compass[instr])) > self.size):
            self.do_expand()
        self.layout[*(self.loc + step + self.size)] = '.'
        self.loc += 2*step

    def do_expand(self):
        self.size += 2
        layout_old = self.layout
        self.layout = np.where(np.expand_dims(y:=np.mod(np.arange(self.size*2+1), 2), 0) 
                               | np.expand_dims(y, 1)==1, 
                               '#', '.')
        self.layout[2:-2,2:-2] = layout_old
        
    def do_reach(self):
        reach = np.pad([[0]], self.size, constant_values=-1)
        while np.any(reach != (reach:=self._exp_reach(reach))):  pass
        self.reach = reach
        
    def _exp_reach(self, reach):
        return np.where((reach == -1) 
                        & (self.layout == '.')
                        & ((z:=np.max(np.stack(((y:=np.pad(reach,1,constant_values=-1))[:-2,1:-1],
                                                y[2:,1:-1], 
                                                y[1:-1,:-2], 
                                                y[1:-1,2:])),
                                      axis=0)) != -1),
                        z+1,
                        reach)
        
    def get_max_nsteps(self):
        return (np.max(self.reach) // 2).item()
        
roommap = RoomMap(rawinput[1:-1])
roommap.do_mapping()
roommap.do_reach()
roommap.get_max_nsteps()

4050

In [4]:
# Part 2
np.sum(roommap.reach[::2, ::2] >= 2000).item()

8564